# Imports

In [ ]:
import matplotlib
matplotlib.use("Agg")
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
from tensorflow.python.keras.applications import densenet
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications import Xception
import numpy as np
import os

#Transfer Learning

This work mainly based on transfer learning where i have used the pre-trained models trained on imagenet except final layer so that it can train our data.

In [ ]:
transferLearningModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

transferModel = transferLearningModel.output
transferModel = Flatten(name="flatten")(transferModel)
transferModel = Dense(1024, activation='relu')(transferModel)
transferModel = Dropout(0.5)(transferModel)
transferModel = Dense(1, activation='sigmoid')(transferModel)

model = Model(inputs=transferLearningModel.input, outputs=transferModel)
for layer in transferLearningModel.layers:
	layer.trainable = False

[Info] loading imagenet weights...
9420800/9406464 [==============================] - 0s 0us/step


Next, we instantiate an object named train_datagen and test_datagen from theImageDataGenerator class, specifying that the pixels will be normalized to the 0-1 range (by dividing by 255)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

Further, Set the directory for going through our data along with that we define standard batch size.

In [ ]:
batch_size = 32
training_directory = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train"     #directory of training data
test_directory = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test"

Then, we use the flow_from_directory method to effectively create the objects that will generate the training and validation minibatches. 
Image size we have define the standard on i.e. (224,224) which is generally an input shape for most of the models.

In [ ]:
train_set = train_datagen.flow_from_directory(training_directory, 
                                                 target_size=(224, 224),
                                                 batch_size=batch_size, 
                                                 class_mode='binary')
test_set = test_datagen.flow_from_directory(test_directory, 
                                            target_size=(224, 224),
                                            batch_size=batch_size, 
                                            class_mode='binary')

Next, some of the parameter is defined. where learning rate i a tested manually from 0.01 to 0.00001. And after running continously i came down to this 0.0001.

In [ ]:
optimizer = Adam(lr=0.0001)
epoch = 10
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history = model.fit_generator(train_set,
                        steps_per_epoch=train_set.samples//batch_size,
                        validation_data=test_set,
                        epochs=epoch,
                        validation_steps=test_set.samples//batch_size)

Next we plot the graph for visualization of accuracy and loss to obseve the perfomance and thus i come to conclusion that 10 spoch is enough.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc', 'val_acc'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Testing our model with some images from test set.

In [ ]:
import cv2
from tensorflow.keras.preprocessing import image
%matplotlib inline
imagepath = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/def_front/cast_def_0_1191.jpeg"
img = cv2.imread(imagepath)
img = cv2.resize(img, (224, 224))
orig = img.copy()
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255

print("[Info] predicting output")
prediction = model.predict(img)
if (prediction<0.5):
    print("def_front")
    cv2.putText(orig, "def_front", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
else:
    print("ok_front")
    cv2.putText(orig, "ok_front", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
plt.imshow(orig)
plt.axis('off')
plt.show()

In [ ]:
%matplotlib inline
imagepath = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/ok_front/cast_ok_0_1092.jpeg"
img = cv2.imread(imagepath)
img = cv2.resize(img, (224, 224))
orig = img.copy()
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255

print("[Info] predicting output")
prediction = model.predict(img)
if (prediction<0.5):
    print("def_front")
    cv2.putText(orig, "def_front", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
else:
    print("ok_front")
    cv2.putText(orig, "ok_front", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
plt.imshow(orig)
plt.axis('off')
plt.show()

# .....................................................................................